In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pprint import pprint

# Psycopg2 is a popular PostgreSQL adapter for the Python programming language. It allows Python code to interact with PostgreSQL databases. 
import psycopg2
from psycopg2 import sql


# Amazon SageMaker and related imports
# import sagemaker
# import sagemaker.amazon.common as smac
# from sagemaker.deserializers import JSONDeserializer
# from sagemaker.serializers import CSVSerializer
# from sagemaker import get_execution_role
# from sagemaker import image_uris
# import boto3  # AWS Python sdk

%matplotlib inline

# Establishes a connection to a PostgreSQL database hosted on Amazon RDS (Relational Database Service) and retrieves data from a tables

In [2]:
#Info for the connection wiht postgres SQL in RDS
PGEND_POINT = 'database-1.cfwmkaw8o6bp.us-east-1.rds.amazonaws.com' #End Point
PGDATABASE_NAME ='heart_attack_prediction_db' #data base name 
PGUSER_NAME = 'postgres'
PGPASSWORD = 'B00TC4MP'

In [3]:
#Defining functions for connection and close connection

def connect():
    conn_string = f"host={PGEND_POINT} port=5432 dbname={PGDATABASE_NAME} user={PGUSER_NAME} password={PGPASSWORD}"
    conn = psycopg2.connect(conn_string)
    print("Connected!")
    
    #Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

#Close connection function definition
def close_connection(conn, cursor):
    conn.commit()
    cursor.close()
    conn.close()
    print("Connection closed.")

In [4]:
#Creting the connection with postgres SQL
conn, cursor = connect()

Connected!


In [5]:
 #SQL SELECT statement that retrieves all columns (*) from the table named heartattackprediction.
query_hat_all = sql.SQL("""
SELECT * FROM heartattackprediction;
""")

In [6]:
#Preparation of the database cursor to execute the SQL query specified by query_hat_all. 
#Once the query is executed, the cursor will hold the result set (if any) 
#returned by the database server.
cur = conn.cursor()
cur.execute(query_hat_all)

In [7]:
#put all the data in heartattackprediction table into a data frame in pandas all the columns name appear
pd_df = pd.DataFrame(cur.fetchall(), columns=['Patient ID', 'Age', 'Sex','Cholesterol', 
                                              'Blood Pressure','Heart Rate','Diabetes','Family History',
                                              'Smoking','Obesity','Alcohol Consumption','Exercise Hours Per Week',
                                              'Diet','Previous Heart Problems','Medication Use','Stress Level',
                                              'Sedentary Hours Per Day','Income','BMI','Triglycerides',
                                              'Physical Activity Days Per Week','Sleep Hours Per Day',
                                              'Country','Continent','Hemisphere','Heart Attack Risk'])
pd_df.head(5)

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [8]:
 #SQL SELECT statement that retrieves all columns (*) from the table named heartattackprediction.
query_hat_all = sql.SQL("""
SELECT * FROM encodedtable2;
""")

In [9]:
#Preparation of the database cursor to execute the SQL query specified by query_hat_all. 
#Once the query is executed, the cursor will hold the result set (if any) 
#returned by the database server.
cur = conn.cursor()
cur.execute(query_hat_all)

In [10]:
#put all the data in heartattackprediction table into a data frame in pandas all the columns name appear
encoded_df = pd.DataFrame(cur.fetchall(), columns=["Patient ID", "Country", "Capital", "Age", "Sex", "Cholesterol",
                                              "Heart Rate", "Diabetes", "Family History", "Smoking", "Obesity",
                                              "Alcohol Consumption", "Exercise Hours Per Week", "Previous Heart Problems",
                                              "Medication Use", "Stress Level", "Sedentary Hours Per Day", "Income", "BMI",
                                              "Triglycerides", "Physical Activity Days Per Week", "Sleep Hours Per Day", "Continent",
                                              "Hemisphere", "Heart Attack Risk", "Systolic Pressure", "Diastolic Pressure", "lat", "long", 
                                              "Diet_Average", "Diet_Healthy", "Diet_Unhealthy"])
encoded_df.head(5)

,Patient ID,Country,Capital,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,...,Continent,Hemisphere,Heart Attack Risk,Systolic Pressure,Diastolic Pressure,lat,long,Diet_Average,Diet_Healthy,Diet_Unhealthy
0,BMW7812,Argentina,Buenos Aires,67,1,208,72,0,0,1,...,South America,1,0,158,88,-34.0,-64.0,1,0,0
1,CZE1114,Canada,Ottawa,21,1,389,98,1,1,1,...,North America,0,0,165,93,60.0,-95.0,0,0,1
2,BNI9906,France,Paris,21,0,324,72,1,0,0,...,Europe,0,0,174,99,46.0,2.0,0,1,0
3,JLN3497,Canada,Ottawa,84,1,383,73,1,1,1,...,North America,0,0,163,100,60.0,-95.0,1,0,0
4,GFO8847,Thailand,Bangkok,66,1,318,93,1,1,1,...,Asia,0,0,91,88,15.0,100.0,0,0,1


In [11]:
# Call this function when you're done with your database operations
close_connection(conn, cursor)

Connection closed.
